# Preprocessing

## Histogram equalization

In [27]:
import cv2 as cv

def histeq(data):
    for slice_index in range(data.shape[2]):
        data[:, :, slice_index] = cv.equalizeHist(data[:, :, slice_index])
    return data

## Skull stripping

In [28]:
import nibabel as nib
from deepbrain import Extractor

def get_data_with_skull_scraping(path, PROB=0.5):
    arr = nib.load(path).get_data()
    ext = Extractor()
    prob = ext.run(arr)
    mask = prob > PROB
    arr = arr * mask
    return arr

Instructions for updating:
non-resource variables are not supported in the long term


## Convert to Uint8

In [29]:
def to_uint8(data):
    data = data.astype(np.float)
    data[data < 0] = 0
    return ((data - data.min()) * 255.0 / data.max()).astype(np.uint8)

## Bias field correction

In [30]:
from __future__ import print_function

import SimpleITK as sitk
import sys
import os

def n4_bias_field_correction(input_image, outputImage=None,
                          shrinkFactor=None, 
                          mask_image=None, 
                          numberOfIterations=None, 
                          number_of_fitting_levels=None):
    
    inputImage = sitk.ReadImage(input_image, sitk.sitkFloat32)
    image = inputImage
    
    if mask_image is not None:
         maskImage = sitk.ReadImage(mask_image, sitk.sitkUint8)
    else:
        maskImage = sitk.OtsuThreshold(inputImage, 0, 1, 200)
    
    if shrinkFactor is not None:
        image = sitk.Shrink(inputImage,
                             [shrinkFactor] * inputImage.GetDimension())
        maskImage = sitk.Shrink(maskImage,
                            [shrinkFactor] * inputImage.GetDimension())
    
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    
    numberFittingLevels = 4
    
    if number_of_fitting_levels is not None:
        numberFittingLevels = number_of_fitting_levels
    
    if numberOfIterations is not None:
        corrector.SetMaximumNumberOfIterations([numberOfIterations] * numberFittingLevels)
    
    corrected_image = corrector.Execute(image, maskImage)
    
    log_bias_field = corrector.GetLogBiasFieldAsImage(inputImage)
    bias_field = inputImage / sitk.Exp( log_bias_field )
    
    if outputImage is not None:
        sitk.WriteImage(corrected_image, outputImage)
        
       
    #TODO: whi this error?
    #RuntimeError: Exception thrown in SimpleITK Show: D:\a\1\sitk\Code\IO\src\sitkShow.cxx:495:
    #sitk::ERROR: No appropriate executable found.
    #if ("SITK_NOSHOW" not in os.environ):
    #    sitk.Show(corrected_image, "N4 Corrected")
    
    return corrected_image, bias_field, log_bias_field

## Exhaustive image registration

In [33]:
from __future__ import print_function
from __future__ import division

import SimpleITK as sitk
import sys
import os
from math import pi

def command_iteration(method):
    if (method.GetOptimizerIteration() == 0):
        print("Scales: ", method.GetOptimizerScales())
    print("{0:3} = {1:7.5f} : {2}".format(method.GetOptimizerIteration(),
                                          method.GetMetricValue(),
                                          method.GetOptimizerPosition()))
    
def exhautive_registration(fixedImageFilter, movingImageFile):
    fixed = sitk.ReadImage(fixedImageFilter, sitk.sitkFloat32)
    moving = sitk.ReadImage(movingImageFile, sitk.sitkFloat32)
    
    R = sitk.ImageRegistrationMethod()
    R.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    
    sample_per_axis = 12
    if fixed.GetDimension() == 2:
        tx = sitk.Euler2DTransform()
        # Set the number of samples (radius) in each dimension, with a
        # default step size of 1.0
        R.SetOptimizerAsExhaustive([sample_per_axis // 2, 0, 0])
        # Utilize the scale to set the step size for each dimension
        R.SetOptimizerScales([2.0 * pi / sample_per_axis, 1.0, 1.0])
    elif fixed.GetDimension() == 3:
        tx = sitk.Euler3DTransform()
        R.SetOptimizerAsExhaustive([sample_per_axis // 2, sample_per_axis // 2,
                                    sample_per_axis // 4, 0, 0, 0])
        R.SetOptimizerScales(
            [2.0 * pi / sample_per_axis, 2.0 * pi / sample_per_axis,
             2.0 * pi / sample_per_axis, 1.0, 1.0, 1.0])
        
    
    # Initialize the transform with a translation and the center of
    # rotation from the moments of intensity.
    tx = sitk.CenteredTransformInitializer(fixed, moving, tx)

    R.SetInitialTransform(tx)

    R.SetInterpolator(sitk.sitkLinear)

    #R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))

    transform = R.Execute(fixed, moving)
    
    #print("-------")
    #print(outTx)
    #print("Optimizer stop condition: {0}"
    #      .format(R.GetOptimizerStopConditionDescription()))
    #print(" Iteration: {0}".format(R.GetOptimizerIteration()))
    #print(" Metric value: {0}".format(R.GetMetricValue()))

    #sitk.WriteTransform(outTx, sys.argv[3])
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(1)
    resampler.SetTransform(transform)
    
    resampled = resampler.Execute(moving)
    
    return transform, resampled

## Convert DICOM batch to nifti

In [24]:
from __future__ import print_function

import sys
import SimpleITK as sitk
import os

def read_batch(in_dir, save_all=True):
    reader = sitk.ImageSeriesReader()
    
    ids = reader.GetGDCMSeriesIDs(in_dir)
    
    images = []
    
    for i in ids:
        dicom_names = reader.GetGDCMSeriesFileNames(in_dir, i)
        reader.SetFileNames(dicom_names)
        reader.MetaDataDictionaryArrayUpdateOn()
        reader.LoadPrivateTagsOn()
        image = reader.Execute()
        images.append(image)
        if save_all == True:
            series_name = reader.GetMetaData(0, '0008|103e')
            sitk.WriteImage(image, '{0}{1}.nii'.format(in_dir, series_name))
    
    #dicom_names = reader.GetGDCMSeriesFileNames(in_dir)
    
    return images

# Tests

In [26]:
imgs = read_batch('C:/Users/StarDust/Desktop/imgs/readfrom/')

In [32]:
corrected, bias, log_bias = n4_bias_field_correction('C:/Users/StarDust/Desktop/imgs/readfrom/T1 FFE3D Tra.nii', 'C:/Users/StarDust/Desktop/imgs/readfrom/corrected_bias_t1.nii')

sitk.WriteImage(bias, 'C:/Users/StarDust/Desktop/imgs/readfrom/bias_t1.nii')
sitk.WriteImage(log_bias, 'C:/Users/StarDust/Desktop/imgs/readfrom/log_bias_t1.nii')

In [34]:
transform, resampled = exhautive_registration('C:/Users/StarDust/Desktop/imgs/filter.nii.gz', 'C:/Users/StarDust/Desktop/imgs/readfrom/corrected_bias_t1.nii')
sitk.WriteTransform(transform, 'C:/Users/StarDust/Desktop/imgs/readfrom/transform.tfm')
sitk.WriteImage(resampled, 'C:/Users/StarDust/Desktop/imgs/readfrom/registered.nii')

In [48]:
fixed = sitk.ReadImage('C:/Users/StarDust/Desktop/imgs/filter.nii.gz', sitk.sitkFloat32)
moving = sitk.ReadImage('C:/Users/StarDust/Desktop/imgs/readfrom/corrected_bias_t1.nii', sitk.sitkFloat32)
simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
simg2 = sitk.Cast(sitk.RescaleIntensity(resampled), sitk.sitkUInt8)
simg3 = sitk.Cast(sitk.RescaleIntensity(moving), sitk.sitkUInt8)
cimg = sitk.Compose(simg1, simg2, simg1//2.+simg2//2.)
sitk.Show( cimg, "ImageRegistrationExhaustive Composition" )
